## About this code
The purpose of this notebook is to show an example of how we can extract information from web pages with some level of accuracy.
I've made this notebook as an example to help you in case you needed to scrape infromation in the future.
Here, we are collecting the speeches of previous presidents of the united states

In [50]:
import os
from os import walk
from bs4 import BeautifulSoup
import requests
import html

### Files structures
The folders exist each named by the name of some of the american presidents
inside each folder, there is an html file "initial.txt" scraped from millercenter.org
containing links for all the speeches of each corresponding president

In [51]:
# parent folder
path = 'presidents-speeches/'

# will contain all the data "president name" : ["speeches links"]
presidential_dictionary_list = []
for (root, dirs, files) in walk(path):
    # go through child folders, skip the first parent folder
    if len(dirs) == 1:
        president_speeches_dict = {}
        # get the name of the president from the file path
        president_name = root.split("/")[1]
        president_speeches_dict['president_name'] = president_name
        speeches_links = []
        file_path = "{}/{}".format(root,'initial.txt')
        # open the "initial.txt" file for each president
        with open(file_path) as html_links_file:
            soup = BeautifulSoup(html_links_file)
            for link in soup.findAll('a'):
                # extract the speeches links from the <a> tags 
                speeches_links.append(link.get('href'))
        president_speeches_dict['speeches_links'] = speeches_links
        presidential_dictionary_list.append(president_speeches_dict)

In [52]:
presidential_dictionary_list

[{'president_name': 'George Washington',
  'speeches_links': ['#',
   '#edit-field-president-target-id--wrapper',
   '#edit-field-speech-date-value-min-wrapper',
   None,
   None,
   '#',
   '/the-presidency/presidential-speeches/december-7-1796-eighth-annual-message-congress',
   '/the-presidency/presidential-speeches/september-19-1796-farewell-address',
   '/the-presidency/presidential-speeches/august-29-1796-talk-cherokee-nation',
   '/the-presidency/presidential-speeches/march-30-1796-message-house-representatives-declining-submit',
   '/the-presidency/presidential-speeches/december-8-1795-seventh-annual-message-congress',
   '/the-presidency/presidential-speeches/july-10-1795-proclamation-pardons-western-pennsylvania',
   '/the-presidency/presidential-speeches/november-19-1794-sixth-annual-message-congress',
   '/the-presidency/presidential-speeches/september-25-1794-proclamation-militia-service',
   '/the-presidency/presidential-speeches/august-7-1794-proclamation-against-opposit

### Fetch speeches from the site
now that we have the links of the speeches, we can fetch them and extract the content
we will save them in files titled by date__title, corresponding to every president

In [53]:
# format speech titles to be appropriate for usage as file names
import re
def file_name_format(text):
    return re.sub('[^A-Za-z0-9\-\_]+', '', text)

In [ ]:
for presidential_dictionary in presidential_dictionary_list:
    print("extracting speeches of president {}".format(presidential_dictionary['president_name']))
    speech_counter = 1
    total_speeches = len(presidential_dictionary['speeches_links'])
    for speech_link in presidential_dictionary['speeches_links']:
        try:
            resp = requests.get("https://millercenter.org/{}".format(speech_link))
        except requests.exceptions.RequestException as e:
            print(e)
        print("speech {}/{}".format(speech_counter,total_speeches))
        speech_counter+=1
        if resp.status_code == 200:
            soup = BeautifulSoup(resp._content)
            president_speech = ''
            # the speech is present inside a <div> with class "view-transcript", as a set of <p> tags
            for transcripts in soup.find_all('div', { "class" : "view-transcript"}): # there is only one view-transcript per page
                ps = transcripts.find_all('p')
                president_speech = ' '.join([p.text for p in ps])
            # get the title and date of every speech, and name the file accordingly
            for title in soup.find_all('h2', { "class" : "presidential-speeches--title"}):
                ttl = title.find('span').text
                date_speech_title = ttl.split(":")
                date = date_speech_title[0].strip()
                speech_title = date_speech_title[1].strip()
                with open(path+presidential_dictionary['president_name']+"/speeches/"+date+"__"+file_name_format(speech_title)+".txt","w+") as out_file:
                    out_file.write(str(president_speech.encode('utf8')))

extracting speeches of president George Washington
speech 1/27
speech 2/27
speech 3/27
speech 4/27
speech 5/27
speech 6/27
speech 7/27
speech 8/27
speech 9/27
speech 10/27
speech 11/27
speech 12/27
speech 13/27
speech 14/27
speech 15/27
speech 16/27
speech 17/27
speech 18/27
speech 19/27
speech 20/27
speech 21/27
speech 22/27
speech 23/27
speech 24/27
speech 25/27
speech 26/27
speech 27/27
extracting speeches of president George W. Bush
speech 1/39
speech 2/39
speech 3/39
speech 4/39
speech 5/39
speech 6/39
speech 7/39
speech 8/39
speech 9/39
speech 10/39
speech 11/39
speech 12/39
speech 13/39
speech 14/39
speech 15/39
speech 16/39
speech 17/39
speech 18/39
speech 19/39
speech 20/39
speech 21/39
speech 22/39
speech 23/39
speech 24/39
speech 25/39
speech 26/39
speech 27/39
speech 28/39
speech 29/39
speech 30/39
speech 31/39
speech 32/39
speech 33/39
speech 34/39
speech 35/39
speech 36/39
speech 37/39
speech 38/39
speech 39/39
extracting speeches of president George H. W. Bush
speech 1/2

### fin
After running this code, all the speeches are be saved inside their corresponding president folder